In [13]:
""" nltk.download('puntk')
nltk.download('stopwords') """
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from natsort import natsorted
import os 
import nltk
import math
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

stop_words = stopwords.words('english')
stop_words.remove('to')
stop_words.remove('in')
stop_words.remove('where')

#READ documents And TOKENIZATION And STEEMING
ps = PorterStemmer()
document_of_terms = []
document_with_stemming = []
file_name = natsorted(os.listdir('../DocumentCollection')) #to sort files in folder
for file in file_name :
    #1.1# Read 10 Documents
    with open(f'../DocumentCollection/{file}', 'r') as f:
        document = f.read()
    #1.2# apply tokenizing
    tokenizing = word_tokenize(document)
    for word in tokenizing : 
        if word not in stop_words :
            #1.3# apply stemming
            stemming = ps.stem(word)
        document_with_stemming.append(stemming)
    document_of_terms.append(document_with_stemming)
    document_with_stemming = []
#print(document_of_terms)

# positional index
document_id = 1
positional_index = {}
for document in document_of_terms:
    for position, term in enumerate(document):
        if term in positional_index:
            positional_index[term][0] = positional_index[term][0] + 1
            if document_id in positional_index[term][1]:
                positional_index[term][1][document_id].append(position)
            else:
                positional_index[term][1][document_id] = [position]
        else:
            positional_index[term] = []
            positional_index[term].append(1)
            positional_index[term].append({})
            positional_index[term][1][document_id] = [position]
    document_id = document_id + 1
sorted_posIndex = dict(sorted(positional_index.items()))
print("POSITIONAL INDEX IS :")
print(sorted_posIndex)
print()
print()

#2.5# phrase query
outer_array = [[] for i in range(10)]
query = "fools fear"
for wordd in query.split():
    word_after_stem = ps.stem(wordd)
    if word_after_stem in positional_index.keys():  
      for key in positional_index[word_after_stem][1].keys():
        if outer_array[key-1] != []:
            if outer_array[key-1][-1] == positional_index[word_after_stem][1][key][0] - 1:
                    outer_array[key-1].append(positional_index[word_after_stem][1][key][0])
        else:
                outer_array[key-1].append(positional_index[word_after_stem][1][key][0])
for position, arr in enumerate(outer_array, start=1):
       if len(arr) == len(query.split()):
            x = position 
            #print(position)

#tf
all_terms = []
for doc in document_of_terms:
    for word in doc:
        all_terms.append(word)
def term_frequency(doc):
    #put each term in dictionary form with value 0
    word_count = dict.fromkeys(all_terms, 0) 
    for word in doc:
        #increment the value of each term in dictionary
        word_count[word] += 1
        # return dictionary of each document
    return word_count
# convert dictionay form into columns form
tf = pd.DataFrame(term_frequency(document_of_terms[0]).values(), index = term_frequency(document_of_terms[0]). keys())
for i in range(1, len(document_of_terms)):
    tf[i] = term_frequency(document_of_terms[i]).values()
#change name of column
tf.columns = ['doc' + str(i) for i in range(1, 11)] 
sorted_tf = tf.sort_index()
print("TERM FREQUENCY IS:")
print(sorted_tf)
print()
print()

#w_tf weighted term frequency 1 + log10(tf)
def w_tf(x):
    if x > 0:
        return math.log10(x) + 1
    else:
        return 0
wt_tf = tf.copy()
for i in range(1, len(document_of_terms) + 1):
    #replace each raw with w_tf
    wt_tf['doc'+ str(i)] =  wt_tf['doc'+str(i)].apply(w_tf) 
sorted_wt_tf = wt_tf.sort_index()
print("WT_TF IS:")
print(sorted_wt_tf)
print()
print()

#2.3# IDF
df_and_IDF = pd.DataFrame(columns=('d_f','idf'))
for i in range(len(tf)):
    doc_freq = tf.iloc[i].values.sum()
    df_and_IDF.loc[i, 'd_f'] = doc_freq
    df_and_IDF.loc[i, 'idf'] = math.log10(10/float(doc_freq))
df_and_IDF.index = tf.index
sorted_df_and_IDF = df_and_IDF.sort_index()
print("DF AND IDF IS :")
print(sorted_df_and_IDF)
print()
print()

# TF-IDF
tf_idf = tf.multiply(df_and_IDF['idf'], axis = 0)
sorted_tf_idf = tf_idf.sort_index()
print("TF_IDF IS :")
print(sorted_tf_idf)
print()
print()

#doc_length
import numpy as np
doc_length = pd.DataFrame()
def document_length(col):
    return np.sqrt(tf_idf[col].apply(lambda x: x**2).sum())
for column in tf_idf.columns:
    doc_length.loc[0, column + '_len'] = document_length(column)
print("DOCUMENT LENGTH IS :")
print(doc_length)
print()
print()

#NORMALIZATION TF-IDF divided by DOC_LENGTH
normalize = pd.DataFrame()
def get_normalize(col, x):
    try:
        return x / doc_length[column + '_len'].values[0]
    except:
        return 0
for column in tf_idf.columns:
    normalize[column] = tf_idf[column].apply(lambda x: get_normalize(column, x))
print("NORMALIZTION")
sorted_normalize = normalize.sort_index()
print(sorted_normalize)
print()
print()

POSITIONAL INDEX IS :
{'angel': [3, {7: [0], 8: [0], 9: [0]}], 'antoni': [3, {1: [0], 2: [0], 6: [0]}], 'brutu': [3, {1: [1], 2: [1], 4: [0]}], 'caeser': [5, {1: [2], 2: [2], 4: [1], 5: [0], 6: [1]}], 'calpurnia': [1, {2: [3]}], 'cleopatra': [1, {1: [3]}], 'fear': [3, {7: [2], 8: [2], 10: [1]}], 'fool': [4, {7: [1], 8: [1], 9: [1], 10: [0]}], 'in': [4, {7: [3], 8: [3], 9: [2], 10: [2]}], 'merci': [5, {1: [4], 3: [0], 4: [2], 5: [1], 6: [2]}], 'rush': [4, {7: [4], 8: [4], 9: [3], 10: [3]}], 'to': [4, {7: [5], 8: [5], 9: [4], 10: [4]}], 'tread': [4, {7: [6], 8: [6], 9: [5], 10: [5]}], 'where': [4, {7: [7], 8: [7], 9: [6], 10: [6]}], 'worser': [4, {1: [5], 3: [1], 4: [3], 5: [2]}]}


TERM FREQUENCY IS:
           doc1  doc2  doc3  doc4  doc5  doc6  doc7  doc8  doc9  doc10
angel         0     0     0     0     0     0     1     1     1      0
antoni        1     1     0     0     0     1     0     0     0      0
brutu         1     1     0     1     0     0     0     0     0      0
caeser 